
Circuit Simulation Example with OBI-One
======================================

This script demonstrates how to run circuit simulations using the OBI-One simulation framework
with [BlueCelluLab](https://github.com/openbraininstitute/BlueCelluLab) as the simulator. Let's get the path to the simulation configuration and circuit configuration file paths and the population name of the circuit that you want to simulate.

For now, we will not use the pre-made `simulation_config.json` (simulation configuration) file instead of the one created by obi-one.

In [ ]:
population_name = "S1nonbarrel_neurons"
circuit_config_path = "../data/tiny_circuits/N_10__top_nodes_dim6__asc/circuit_config.json"
simulation_config_path = "../data/tiny_circuits/N_10__top_nodes_dim6__asc/simulation_config.json"

Get the path to the mod files and compile the mod files. 

In [ ]:
circuit_folder = "../data/tiny_circuits/N_10__top_nodes_dim6__asc"
!nrnivmodl {circuit_folder}/mod

Import required modules from obi-one

In [ ]:

from obi_one.scientific.simulation.simulations import Simulation
# The imports below will be used when simulation config is created using obi-one
from obi_one.scientific.simulation.stimulus import ConstantCurrentClampSomaticStimulus
from obi_one.scientific.simulation.recording import SomaVoltageRecording
from obi_one.scientific.simulation.timestamps import RegularTimestamps
from obi_one.scientific.circuit.neuron_sets import IDNeuronSet
from obi_one.core.tuple import NamedTuple
from obi_one.scientific.unions.unions_neuron_sets import NeuronSetReference
from obi_one.scientific.unions.unions_timestamps import TimestampsReference

Create the `Simulation` 

In [ ]:
# Create regular timestamps with start, end, and dt
timestamps = RegularTimestamps(
    number_of_repetitions=1, 
    interval=100.0,  # ms
    start_time=0.0,  # ms
    end_time=100.0,  # ms
    dt=0.1,          # ms
    simulation_level_name="timestamps_1"
)

# Create a NamedTuple with the neuron IDs
neuron_ids = NamedTuple(name="neuron_ids", elements=[1, 2, 3])  # List of cell IDs to include

# Create neuron set using IDNeuronSet
neuron_set = IDNeuronSet(
    population=population_name,
    neuron_ids=neuron_ids,
    simulation_level_name="neuron_set_1"
)

# First, create the timestamps reference
timestamps_ref = TimestampsReference(block=timestamps, block_name="timestamps_1")

# Then create the stimulus with the reference
stimulus = ConstantCurrentClampSomaticStimulus(
    timestamps=timestamps_ref,
    delay=10.0,  # ms
    duration=50.0,  # ms
    amplitude=0.1,  # nA
    neuron_set=NeuronSetReference(block=neuron_set, block_name="neuron_set_1"),
    simulation_level_name="stimulus_1"
)
# Create recordings (example with voltage recording)
recording = SomaVoltageRecording(
    start_time=0.0,
    end_time=100.0,
    dt=0.1,
    neuron_set=NeuronSetReference(block=neuron_set, block_name="neuron_set_1"),
    simulation_level_name="voltage_recording_1"
)

from obi_one.scientific.circuit.circuit import Circuit

# First create the circuit object
circuit = Circuit(
    name="N_10__top_nodes_dim6__asc",
    path=circuit_config_path,
    node_population=population_name,
)

from obi_one.core.info import Info
# Create info object
simulation_info = Info(
    name="N_10__top_nodes_dim6__asc Simulation",
    description="A test simulation with a small microcircuit",
    author="Darshan Mandge",
    version="0.0.1"
)

# Then use it in the simulation initialization
simulation = Simulation(
    name="simulation_1",
    info=simulation_info,
    timestamps={"timestamps_1": timestamps},  # Make sure this matches the block_name
    stimuli={"stimulus_1": stimulus},
    recordings={"voltage_recording_1": recording},
    neuron_sets={"neuron_set_1": neuron_set},
    initialize={
        "circuit": circuit,
        "simulation_length": 100.0,  # ms
        "node_set": NeuronSetReference(block=neuron_set, block_name="neuron_set_1"),
        "timestep": 0.025  # ms
    }
)

If you want to generate configuration uncomment the cell below.

In [ ]:
# 6. Generate the simulation configuration
# simulation.generate()

Provide a list of gid (cells ids) from the circuit to simulate

In [ ]:

gids_to_simulate = [0,1,2,3,4,5,6,7,8,9]

We pass `cell_ids` as list of of gid `(population_name, gid)`

In [ ]:
# create for each gid (population_name, gid)
# cell_ids is list of these tuples
cell_ids = []
for gid in gids_to_simulate:
    cell_ids.append((population_name, gid))
    

Run circuit simulation using BlueCelluLab backend. In future, we will support Neurodamus backend as well. This will run a SONATA simulation and save voltage traces for the specified cells.  

In [ ]:
# Run the simulation
simulation.run(
    simulation_config=simulation_config_path,
    cell_ids=cell_ids,
    simulator="bluecellulab", 
    results_dir=None, # optional. Else, will use simulation_config's manifest
    save_nwb=True)

### Spike Report Analysis using BluePySnap

We will use the example [notebook](https://github.com/openbraininstitute/snap/blob/master/doc/source/notebooks/06_spike_reports.ipynb) from BluePySnap to analyse the spike report generated by the circuit simulation.

In [ ]:
import bluepysnap

simulation = bluepysnap.Simulation(simulation_config_path)
spikes = simulation.spikes
print(
    spikes.time_start,
    spikes.time_stop, 
    spikes.dt
)
print(spikes.population_names)

In [ ]:
spike_pop = spikes['S1nonbarrel_neurons']
print(type(spike_pop))

node_population = spike_pop.nodes
print(f'{node_population.name}: {type(node_population)}')

ids = spike_pop.node_ids
node_population.get(ids, properties=['layer','synapse_class','x','y','z']).head()

In [ ]:
filtered = spikes.filter(group={'layer':'6'}, t_start=spikes.time_start, t_stop=spikes.time_stop)
filtered.report.head()

In [ ]:
filtered.raster();

In [ ]:
filtered.firing_rate_histogram();

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

ax = plt.gca()
ax.set_xlabel("Time [ms]")
ax.set_ylabel("PSTH [Hz]")
ax.set_title(f"PSTH for group: {filtered.group}")

times = filtered.report.index

time_start = np.min(times)
time_stop = np.max(times)

# heuristic for a nice bin size (~100 spikes per bin on average)
time_binsize = min(50.0, (time_stop - time_start) / ((len(times) / 100.0) + 1.0))

bins = np.append(np.arange(time_start, time_stop, time_binsize), time_stop)
hist, bin_edges = np.histogram(times, bins=bins)
node_count = len(simulation.circuit.nodes.ids(filtered.group))  # Get length of node ids for whole `group`
freq = 1.0 * hist / node_count / (0.001 * time_binsize)

# use the middle of the bins instead of the start of the bin
ax.plot(0.5 * (bin_edges[1:] + bin_edges[:-1]), freq, label="PSTH", drawstyle="steps-mid");

In [ ]:
spikes.filter().raster();

In [ ]:
spikes.filter().raster(y_axis='etype');

In [ ]:
spikes.filter().isi(binsize=100);

### Soma Report Analysis

Let's [load](https://github.com/openbraininstitute/snap/blob/master/doc/source/notebooks/07_frame_reports.ipynb) the soma report using BluePySnap and plot it.

In [ ]:
simulation.reports

In [ ]:
soma_report = simulation.reports['SomaVoltRec']

In [ ]:
print(
    soma_report.time_start, 
    soma_report.time_stop, 
    soma_report.dt,
    soma_report.time_units
)  # Gives a warning in case the dt differs from simulation.dt

In [ ]:
soma_report.population_names

In [ ]:
soma_pop = soma_report['S1nonbarrel_neurons']
print(type(soma_pop))

In [ ]:
node_population = soma_pop.nodes
print(f'{node_population.name}: {type(node_population)}')

In [ ]:
ids = soma_pop.node_ids
node_population.get(ids, properties=['layer','synapse_class','x','y','z']).head()

In [ ]:
filtered = soma_report.filter(group={'layer':'6'}, t_start= soma_report.time_start, t_stop= soma_report.time_stop)
filtered.report.head()

In [ ]:
filtered.trace();

In [ ]:
soma_report.filter().trace();